In [6]:
import pandas as pd
import numpy as np
from tpot import TPOTClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score as accuracy
from sklearn.metrics import recall_score as recall
from sklearn.metrics import precision_score as precision
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split

In [7]:
df_normal = pd.read_csv("archive/ptbdb_normal.csv", header = None)
df_abnormal = pd.read_csv("archive/ptbdb_abnormal.csv", header = None)
df = pd.concat([df_normal, df_abnormal])
print(df.shape)
# Separating features and target 

X = np.array(df[df.columns[:-1]])
print(f'The feature space is of shape: {X.shape}')
Y= np.array(df[187])
print(f'The target is of shape: {Y.shape}')

(14552, 188)
The feature space is of shape: (14552, 187)
The target is of shape: (14552,)


In [8]:
Xtrain, Xtest, ytrain, ytest =train_test_split(X, Y, test_size=0.2, random_state=42)

In [10]:
# TPOT for classification

### START CODE HERE ###
# Instantiate and train a TPOT auto-ML classifier
# Set generations to 5, population_size to 40, and verbosity to 2 (so you can see each generation's performance)
tpot = TPOTClassifier(generations=5, population_size=40, verbosity=2, random_state=42)
tpot.fit(Xtrain, ytrain)
# Evaluate the classifier on the test data
# By default, the scoring function is accuracy
print(tpot.score(Xtest, ytest))
### END CODE HERE ###
tpot.export('tpotResults2.py')

Optimization Progress:   0%|          | 0/240 [00:00<?, ?pipeline/s]


Generation 1 - Current best internal CV score: 0.9744867862112111

Generation 2 - Current best internal CV score: 0.9762910943593329

Generation 3 - Current best internal CV score: 0.9762910943593329

Generation 4 - Current best internal CV score: 0.9769779000470683

Generation 5 - Current best internal CV score: 0.9769779000470683

Best pipeline: GradientBoostingClassifier(BernoulliNB(CombineDFs(input_matrix, input_matrix), alpha=0.001, fit_prior=False), learning_rate=0.5, max_depth=8, max_features=0.35000000000000003, min_samples_leaf=20, min_samples_split=17, n_estimators=100, subsample=0.6000000000000001)
0.9804190999656476


In [12]:
## Creating and running the model suggested by tpot
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import BernoulliNB
from sklearn.pipeline import make_pipeline, make_union
from tpot.builtins import StackingEstimator
from tpot.export_utils import set_param_recursive
from sklearn.preprocessing import FunctionTransformer
from copy import copy
exported_pipeline = make_pipeline(
    make_union(
        FunctionTransformer(copy),
        FunctionTransformer(copy)
    ),
    StackingEstimator(estimator=BernoulliNB(alpha=0.001, fit_prior=False)),
    GradientBoostingClassifier(learning_rate=0.5, max_depth=8, max_features=0.35000000000000003, min_samples_leaf=20, min_samples_split=17, n_estimators=100, subsample=0.6000000000000001)
)
# Fix random state for all the steps in exported pipeline
set_param_recursive(exported_pipeline.steps, 'random_state', 42)

exported_pipeline.fit(Xtrain, ytrain)
yfit = exported_pipeline.predict(Xtest)

In [13]:
cm= confusion_matrix(ytest,yfit )
acc  = accuracy(ytest,yfit)
rec  = recall(ytest,yfit )
prec = precision(ytest,yfit )
f1   = f1_score(ytest,yfit )
print(f'Accuracy = {acc}, Precision = {prec}, Recall = {rec}, F1-score = {f1}')
print('Confusion Matrix is:')
print(cm)

Accuracy = 0.9804190999656476, Precision = 0.9814108674928503, Recall = 0.9913336543090997, F1-score = 0.9863473053892216
Confusion Matrix is:
[[ 795   39]
 [  18 2059]]
